In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.7.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

In [3]:
def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [4]:
def move_forward(vehicle):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    vehicle.apply_control(control)
 

In [5]:
def check_lateral_collision(azimuth, depth):
    return abs(depth * math.sin(azimuth)) < vehicle_lateral_dimension

In [6]:


def check_vertical_collision(altitude, depth):
    return abs(depth * math.sin(altitude)) < vehicle_vertical_dimension 

In [7]:
min_ttc = float('inf')
counter = 0

def forward_collision_callback(radar_data):
        global min_ttc, counter
        obastacle_detected = False
        for detection in radar_data:
            if check_lateral_collision(detection.azimuth, detection.depth) and check_vertical_collision(detection.altitude, detection.depth) and detection.velocity < velocity_threshold:
                  ttc = detection.depth / abs(detection.velocity)
                  obastacle_detected = True
                  if ttc < min_ttc:
                    min_ttc = ttc   

        if not obastacle_detected:
            counter += 1
            if counter > free_from_obstacle_counter:
              counter = 0
              min_ttc = float('inf')    
        else: 
            counter = 0      

In [8]:
def create_radar(vehicle):
    rad_bp = world.get_blueprint_library().find('sensor.other.radar')
    rad_bp.set_attribute('horizontal_fov', str(35))
    rad_bp.set_attribute('vertical_fov', str(20))
    rad_bp.set_attribute('range', str(250))
    rad_location = carla.Location(x=2.0, z=1.0)
    rad_rotation = carla.Rotation()
    rad_transform = carla.Transform(rad_location,rad_rotation)
    return world.spawn_actor(rad_bp,rad_transform,attach_to=vehicle, attachment_type=carla.AttachmentType.Rigid)


In [16]:
vehicle_vertical_dimension = 0.8
vehicle_lateral_dimension = 0.5
ttc_warning_threshold = 5.0
ttc_stop_threshold = 2.0
throttle = 0.7
free_from_obstacle_counter = 10
velocity_threshold = 0

try:
    vehicle = spawn_vehicle()
    radar = create_radar(vehicle)
    #vehicle.set_autopilot(True)
    radar.listen(lambda radar_data: forward_collision_callback(radar_data))
    while True:
        if min_ttc < ttc_stop_threshold:
            stop_vehicle(vehicle)
            print("Emergency braking activated!")  
        elif min_ttc < ttc_warning_threshold:    
            print("Emergency warning!")  
        else: 
            move_forward(vehicle)
        time.sleep(0.05)

finally:
    vehicle.destroy()
    

Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency warning!
Emergency wa

KeyboardInterrupt: 